In [1]:
%cd ../../../../

/home/users/dmoreno2016/ASTROMER/astromer_pe/astromer


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

import os

from src.data.record import deserialize
from src.layers.positional import PositionalEmbedding

%load_ext autoreload
%autoreload 2

2023-10-19 12:55:32.447101: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 12:55:33.822073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
path_dataset = './data/records/alcock/fold_0/alcock_20/train'

rec_paths = []
for folder in os.listdir(path_dataset):
    if folder.endswith('.csv'):
        continue
    for x in os.listdir(os.path.join(path_dataset, folder)):
        rec_paths.append(os.path.join(path_dataset, folder, x))

dataset = tf.data.TFRecordDataset(rec_paths)    
dataset = dataset.map(deserialize)

2023-10-19 12:55:39.899002: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
for lc_info in dataset:
    lc_data = lc_info['input']
    break

lc_data.shape

TensorShape([635, 3])

In [5]:
toy_lc_data = lc_data[0:7,:]
toy_lc_data

<tf.Tensor: shape=(7, 3), dtype=float32, numpy=
array([[ 4.8917543e+04, -5.0609999e+00,  3.7999999e-02],
       [ 4.8918762e+04, -5.0390000e+00,  3.4000002e-02],
       [ 4.8919555e+04, -5.0730000e+00,  8.3999999e-02],
       [ 4.8920539e+04, -5.1550002e+00,  2.8999999e-02],
       [ 4.8927508e+04, -5.3639998e+00,  6.1999999e-02],
       [ 4.8928543e+04, -5.0679998e+00,  4.3000001e-02],
       [ 4.8929488e+04, -4.9740000e+00,  6.4000003e-02]], dtype=float32)>

In [6]:
toy_lc_time = toy_lc_data[:,0]
toy_lc_flux = toy_lc_data[:,1]

toy_lc_time = tf.expand_dims(tf.expand_dims(toy_lc_time, 1), 0)
toy_lc_flux = tf.expand_dims(tf.expand_dims(toy_lc_flux, 1), 0)

toy_lc_time

<tf.Tensor: shape=(1, 7, 1), dtype=float32, numpy=
array([[[48917.543],
        [48918.76 ],
        [48919.555],
        [48920.54 ],
        [48927.508],
        [48928.543],
        [48929.49 ]]], dtype=float32)>

In [7]:
d_model      = 6
base         = 1000
c            = 1
use_mjd      = True
pe_trainable = False
initializer  = 'pe_astromer' 
min_period   = 0.01
max_period   = 1825  # 5.*365

pos_embedding = PositionalEmbedding(d_model, base=1000, c=1, use_mjd=True, pe_trainable=False, 
			  	                    initializer='pe_astromer', min_period=0.01, max_period=5.*365)

pe_t = pos_embedding(toy_lc_time)
pe_t

<tf.Tensor: shape=(1, 7, 6), dtype=float32, numpy=
array([[[ 0.19498317,  0.9931323 , -0.2899213 ,  0.31996304,
         -0.79138386, -0.7297554 ],
        [-0.8534188 ,  0.96417606, -0.40430143,  0.28330478,
         -0.78388166, -0.72711056],
        [-0.97021437,  0.8680906 , -0.47531015,  0.25910154,
         -0.77893037, -0.7253903 ],
        [-0.33512622,  0.6748755 , -0.5596383 ,  0.22891594,
         -0.77273047, -0.7232442 ],
        [ 0.33709115, -0.9942284 , -0.9610362 ,  0.01062681,
         -0.7266476 , -0.7078535 ],
        [ 0.9816599 , -0.9759721 , -0.98445475, -0.02208604,
         -0.7195014 , -0.7055324 ],
        [ 0.7293001 , -0.86857057, -0.996654  , -0.05197164,
         -0.7128991 , -0.7034098 ]]], dtype=float32)>

In [8]:
import six
import math

from typing import Optional

Initializer = tf.keras.initializers.Initializer

def assert_rank(tensor, expected_rank, name=None):
  """Raises an exception if the tensor rank is not of the expected rank.

  Args:
    tensor: A tf.Tensor to check the rank of.
    expected_rank: Python integer or list of integers, expected rank.
    name: Optional name of the tensor for the error message.

  Raises:
    ValueError: If the expected shape doesn't match the actual shape.
  """
  expected_rank_dict = {}
  if isinstance(expected_rank, six.integer_types):
    expected_rank_dict[expected_rank] = True
  else:
    for x in expected_rank:
      expected_rank_dict[x] = True

  actual_rank = tensor.shape.ndims
  if actual_rank not in expected_rank_dict:
    raise ValueError(
        "For the tensor `%s`, the actual tensor rank `%d` (shape = %s) is not "
        "equal to the expected tensor rank `%s`" %
        (name, actual_rank, str(tensor.shape), str(expected_rank)))

def get_shape_list(tensor, expected_rank=None, name=None):
  """Returns a list of the shape of tensor, preferring static dimensions.

  Args:
    tensor: A tf.Tensor object to find the shape of.
    expected_rank: (optional) int. The expected rank of `tensor`. If this is
      specified and the `tensor` has a different rank, and exception will be
      thrown.
    name: Optional name of the tensor for the error message.

  Returns:
    A list of dimensions of the shape of tensor. All static dimensions will
    be returned as python integers, and dynamic dimensions will be returned
    as tf.Tensor scalars.
  """
  if expected_rank is not None:
    assert_rank(tensor, expected_rank, name)

  shape = tensor.shape.as_list()

  non_static_indexes = []
  for (index, dim) in enumerate(shape):
    if dim is None:
      non_static_indexes.append(index)

  if not non_static_indexes:
    return shape

  dyn_shape = tf.shape(tensor)
  for index in non_static_indexes:
    shape[index] = dyn_shape[index]
  return shape

def _relative_position_bucket(relative_position,
                              bidirectional=True,
                              num_buckets=32,
                              max_distance=128):
  """Translate relative position to a bucket number for relative attention.

  The relative position is defined as memory_position - query_position, i.e.
  the distance in tokens from the attending position to the attended-to
  position.

  If `bidirectional=False`, then positive relative positions are invalid.

  We use smaller buckets for small absolute relative_position and larger
  buckets for larger absolute relative_positions.

  All relative positions >=max_distance map to the same bucket.

  All relative positions <=-max_distance map to the same bucket.

  This should allow for more graceful generalization to longer sequences
  than the model has been trained on.

  Args:
    relative_position: An int32 Tensor
    bidirectional: A boolean - whether the attention is bidirectional
    num_buckets: An integer
    max_distance: An integer

  Returns:
    A Tensor with the same shape as relative_position, containing int32
    values in the range [0, num_buckets)
  """
  ret = 0
  n = -relative_position
  if bidirectional:
    num_buckets //= 2
    ret += tf.cast(tf.math.less(n, 0), tf.int32) * num_buckets
    n = tf.math.abs(n)
    print('num_buckets: {}'.format(num_buckets))
    print('ret:\n{}'.format(ret))
  else:
    n = tf.math.maximum(n, 0)
  
  print('n:\n{}'.format(n))

  # now n is in the range [0, inf)
  max_exact = num_buckets // 2
  is_small = tf.math.less(n, max_exact)
  print('max_exact: {}'.format(max_exact))
  print('is_small:\n{}'.format(is_small))

  val_if_large = max_exact + tf.dtypes.cast(
      tf.math.log(tf.cast(n, tf.float32) / max_exact) /
      math.log(max_distance / max_exact) * (num_buckets - max_exact),
      tf.int32,
  )
  print('val_if_large:\n{}'.format(val_if_large))
  val_if_large = tf.math.minimum(val_if_large, num_buckets - 1)
  print('val_if_large:\n{}'.format(val_if_large))
  ret += tf.where(is_small, n, val_if_large)
  print('ret:\n{}'.format(ret))
  return ret


@tf.keras.utils.register_keras_serializable(package="Text")
class RelativePositionBias(tf.keras.layers.Layer):
  """Relative position embedding via per-head bias in T5 style.

  Reference implementation in MeshTF:
  https://github.com/tensorflow/mesh/blob/master/mesh_tensorflow/transformer/transformer_layers.py#L1000

  This layer implements the relative position bias used in "Exploring the Limits
  of Transfer Learning with a Unified Text-to-Text Transformer"
  (https://arxiv.org/abs/1910.10683)
  """

  def __init__(self,
               num_heads: int,
               relative_attention_num_buckets: int = 32,
               relative_attention_max_distance: int = 128,
               bidirectional: bool = True,
               embeddings_initializer: Optional[Initializer] = None,
               **kwargs):
    super().__init__(**kwargs)
    self.num_heads = num_heads
    self.relative_attention_num_buckets = relative_attention_num_buckets
    self.bidirectional = bidirectional
    self.relative_attention_max_distance = relative_attention_max_distance
    if embeddings_initializer:
      self._embed_init = embeddings_initializer
    else:
      self._embed_init = tf.keras.initializers.TruncatedNormal(stddev=1.0)
    with tf.name_scope(self.name):
      self._relative_attention_bias = self.add_weight(
          "rel_embedding",
          shape=[self.relative_attention_num_buckets, self.num_heads],
          initializer=self._embed_init,
          dtype=self.dtype,
          trainable=True)

  def get_config(self):
    config = {
        "num_heads":
            self.num_heads,
        "relative_attention_num_buckets":
            self.relative_attention_num_buckets,
        "relative_attention_max_distance":
            self.relative_attention_max_distance,
        "bidirectional":
            self.bidirectional,
        "embeddings_initializer":
            tf.keras.initializers.serialize(self._embed_init),
    }
    base_config = super().get_config()
    return dict(list(base_config.items()) + list(config.items()))

  def call(self, query: tf.Tensor, key: tf.Tensor):
    """Implements the forward pass.

    Args:
      query: query input tensor shape [batch, query length, hidden size].
      key: key input tensor shape [batch, key length, hidden size].

    Returns:
      A tensor in shape of [batch, heads, query length, key length].
    """
    print('**************** RELATIVE POSITION BIAS ********************')
    batch_size, qlen = get_shape_list(query)[:2]
    print('batch_size: {}'.format(batch_size))
    print('qlen: {}'.format(qlen))

    klen = get_shape_list(key)[1]
    print('klen: {}'.format(klen))

    context_position = tf.range(qlen)[:, None]
    memory_position = tf.range(klen)[None, :]
    relative_position = memory_position - context_position
    print('context_position: {}'.format(context_position))
    print('memory_position: {}'.format(memory_position))
    print('relative_position: {}'.format(relative_position))

    print('**************** Relative position bucket ********************')

    rp_bucket = _relative_position_bucket(
        relative_position,
        bidirectional=self.bidirectional,
        num_buckets=self.relative_attention_num_buckets,
        max_distance=self.relative_attention_max_distance)
    
    print('**************** embedding_lookup ********************')

    print('self._relative_attention_bias: ', self._relative_attention_bias)
    print('rp_bucket: ', rp_bucket)

    values = tf.nn.embedding_lookup(self._relative_attention_bias, rp_bucket)
    print('values: ', values)
    values = tf.expand_dims(
        tf.transpose(values, [2, 0, 1]),
        axis=0)  # shape (1, num_heads, qlen, klen)
    print('values: ', values)
    values = tf.tile(values, [batch_size, 1, 1, 1])
    print('values: ', values)
    
    return values

In [193]:
wq = tf.keras.layers.Dense(d_model, name='WQ')
wk = tf.keras.layers.Dense(d_model, name='WK')

query = wq(toy_lc_flux)
key = wk(toy_lc_flux)
query

<tf.Tensor: shape=(1, 7, 6), dtype=float32, numpy=
array([[[ 4.2836976 ,  3.1413937 ,  1.1413734 , -0.9834503 ,
          1.6195555 , -0.44428775],
        [ 4.2650766 ,  3.1277382 ,  1.1364119 , -0.97917527,
          1.6125154 , -0.44235647],
        [ 4.293854  ,  3.1488423 ,  1.1440797 , -0.98578215,
          1.6233957 , -0.4453412 ],
        [ 4.3632603 ,  3.1997402 ,  1.1625726 , -1.0017164 ,
          1.6496363 , -0.4525397 ],
        [ 4.5401607 ,  3.3294675 ,  1.2097069 , -1.0423291 ,
          1.7165177 , -0.47088706],
        [ 4.2896223 ,  3.1457386 ,  1.142952  , -0.98481053,
          1.6217955 , -0.44490224],
        [ 4.2100596 ,  3.0873923 ,  1.1217529 , -0.9665445 ,
          1.591715  , -0.43665034]]], dtype=float32)>

In [194]:
num_heads = d_model
relative_attention_num_buckets = 4*2
bidirectional = True
relative_attention_max_distance = 4

rel_pos_bias = RelativePositionBias(num_heads, relative_attention_num_buckets,
                                    bidirectional, relative_attention_max_distance)

values = rel_pos_bias(query, key)
values

**************** RELATIVE POSITION BIAS ********************
batch_size: 1
qlen: 7
klen: 7
context_position: [[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]]
memory_position: [[0 1 2 3 4 5 6]]
relative_position: [[ 0  1  2  3  4  5  6]
 [-1  0  1  2  3  4  5]
 [-2 -1  0  1  2  3  4]
 [-3 -2 -1  0  1  2  3]
 [-4 -3 -2 -1  0  1  2]
 [-5 -4 -3 -2 -1  0  1]
 [-6 -5 -4 -3 -2 -1  0]]
**************** Relative position bucket ********************
num_buckets: 4
ret:
[[0 4 4 4 4 4 4]
 [0 0 4 4 4 4 4]
 [0 0 0 4 4 4 4]
 [0 0 0 0 4 4 4]
 [0 0 0 0 0 4 4]
 [0 0 0 0 0 0 4]
 [0 0 0 0 0 0 0]]
n:
[[0 1 2 3 4 5 6]
 [1 0 1 2 3 4 5]
 [2 1 0 1 2 3 4]
 [3 2 1 0 1 2 3]
 [4 3 2 1 0 1 2]
 [5 4 3 2 1 0 1]
 [6 5 4 3 2 1 0]]
max_exact: 2
is_small:
[[ True  True False False False False False]
 [ True  True  True False False False False]
 [False  True  True  True False False False]
 [False False  True  True  True False False]
 [False False False  True  True  True False]
 [False False False False  True  True  True]
 [False False

InvalidArgumentError: Exception encountered when calling layer 'relative_position_bias_9' (type RelativePositionBias).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[6,0] = -1 is not in [0, 8) [Op:ResourceGather] name: 

Call arguments received by layer 'relative_position_bias_9' (type RelativePositionBias):
  • query=tf.Tensor(shape=(1, 7, 6), dtype=float32)
  • key=tf.Tensor(shape=(1, 7, 6), dtype=float32)

In [10]:
toy_lc_time - tf.transpose(toy_lc_time, perm=[0,2,1])

<tf.Tensor: shape=(1, 7, 7), dtype=float32, numpy=
array([[[  0.        ,  -1.21875   ,  -2.0117188 ,  -2.9960938 ,
          -9.964844  , -11.        , -11.9453125 ],
        [  1.21875   ,   0.        ,  -0.79296875,  -1.7773438 ,
          -8.746094  ,  -9.78125   , -10.7265625 ],
        [  2.0117188 ,   0.79296875,   0.        ,  -0.984375  ,
          -7.953125  ,  -8.988281  ,  -9.933594  ],
        [  2.9960938 ,   1.7773438 ,   0.984375  ,   0.        ,
          -6.96875   ,  -8.003906  ,  -8.949219  ],
        [  9.964844  ,   8.746094  ,   7.953125  ,   6.96875   ,
           0.        ,  -1.0351562 ,  -1.9804688 ],
        [ 11.        ,   9.78125   ,   8.988281  ,   8.003906  ,
           1.0351562 ,   0.        ,  -0.9453125 ],
        [ 11.9453125 ,  10.7265625 ,   9.933594  ,   8.949219  ,
           1.9804688 ,   0.9453125 ,   0.        ]]], dtype=float32)>

### Debbuging pos rel

In [124]:
batch_size, qlen = get_shape_list(query)[:2]
print('batch_size: {}'.format(batch_size))
print('qlen: {}'.format(qlen))

batch_size: 1
qlen: 7


In [125]:
klen = get_shape_list(key)[1]
print('klen: {}'.format(klen))

klen: 7


<tf.Tensor: shape=(1, 1, 7), dtype=float32, numpy=
array([[[48917.543, 48918.76 , 48919.555, 48920.54 , 48927.508,
         48928.543, 48929.49 ]]], dtype=float32)>

In [126]:
context_position = tf.range(qlen)[:, None]
context_position

<tf.Tensor: shape=(7, 1), dtype=int32, numpy=
array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6]], dtype=int32)>

In [127]:
memory_position = tf.range(klen)[None, :]
memory_position

<tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[0, 1, 2, 3, 4, 5, 6]], dtype=int32)>

In [128]:
relative_position = memory_position - context_position
relative_position

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6],
       [-1,  0,  1,  2,  3,  4,  5],
       [-2, -1,  0,  1,  2,  3,  4],
       [-3, -2, -1,  0,  1,  2,  3],
       [-4, -3, -2, -1,  0,  1,  2],
       [-5, -4, -3, -2, -1,  0,  1],
       [-6, -5, -4, -3, -2, -1,  0]], dtype=int32)>

In [131]:
bidirectional = True
relative_attention_num_buckets = num_buckets = 4
relative_attention_max_distance = max_distance = 4

In [117]:
rp_bucket = _relative_position_bucket(
    relative_position,
    bidirectional=bidirectional,
    num_buckets=relative_attention_num_buckets,
    max_distance=relative_attention_max_distance)

rp_bucket

num_buckets: 2
ret:
[[0 2 2 2 2 2 2]
 [0 0 2 2 2 2 2]
 [0 0 0 2 2 2 2]
 [0 0 0 0 2 2 2]
 [0 0 0 0 0 2 2]
 [0 0 0 0 0 0 2]
 [0 0 0 0 0 0 0]]
n:
[[0 1 2 3 4 5 6]
 [1 0 1 2 3 4 5]
 [2 1 0 1 2 3 4]
 [3 2 1 0 1 2 3]
 [4 3 2 1 0 1 2]
 [5 4 3 2 1 0 1]
 [6 5 4 3 2 1 0]]
max_exact: 1
is_small:
[[ True False False False False False False]
 [False  True False False False False False]
 [False False  True False False False False]
 [False False False  True False False False]
 [False False False False  True False False]
 [False False False False False  True False]
 [False False False False False False  True]]
val_if_large:
[[-2147483647           1           1           1           1           2
            2]
 [          1 -2147483647           1           1           1           1
            2]
 [          1           1 -2147483647           1           1           1
            1]
 [          1           1           1 -2147483647           1           1
            1]
 [          1           1   

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[0, 3, 3, 3, 3, 3, 3],
       [1, 0, 3, 3, 3, 3, 3],
       [1, 1, 0, 3, 3, 3, 3],
       [1, 1, 1, 0, 3, 3, 3],
       [1, 1, 1, 1, 0, 3, 3],
       [1, 1, 1, 1, 1, 0, 3],
       [1, 1, 1, 1, 1, 1, 0]], dtype=int32)>

In [133]:
ret = 0
n = -relative_position
n

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[ 0, -1, -2, -3, -4, -5, -6],
       [ 1,  0, -1, -2, -3, -4, -5],
       [ 2,  1,  0, -1, -2, -3, -4],
       [ 3,  2,  1,  0, -1, -2, -3],
       [ 4,  3,  2,  1,  0, -1, -2],
       [ 5,  4,  3,  2,  1,  0, -1],
       [ 6,  5,  4,  3,  2,  1,  0]], dtype=int32)>

In [135]:
if bidirectional:
    num_buckets //= 2
    ret += tf.cast(tf.math.less(n, 0), tf.int32) * num_buckets
    n = tf.math.abs(n)
    print('num_buckets: {}'.format(num_buckets))
    print('ret:\n{}'.format(ret))
else:
    n = tf.math.maximum(n, 0)
    
print('n:\n{}'.format(n))

num_buckets: 2
ret:
[[0 2 2 2 2 2 2]
 [0 0 2 2 2 2 2]
 [0 0 0 2 2 2 2]
 [0 0 0 0 2 2 2]
 [0 0 0 0 0 2 2]
 [0 0 0 0 0 0 2]
 [0 0 0 0 0 0 0]]
n:
[[0 1 2 3 4 5 6]
 [1 0 1 2 3 4 5]
 [2 1 0 1 2 3 4]
 [3 2 1 0 1 2 3]
 [4 3 2 1 0 1 2]
 [5 4 3 2 1 0 1]
 [6 5 4 3 2 1 0]]


In [88]:
# now n is in the range [0, inf)
max_exact = num_buckets // 2
max_exact

1

In [89]:
is_small = tf.math.less(n, max_exact)
is_small

<tf.Tensor: shape=(7, 7), dtype=bool, numpy=
array([[ True, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False],
       [False, False,  True, False, False, False, False],
       [False, False, False,  True, False, False, False],
       [False, False, False, False,  True, False, False],
       [False, False, False, False, False,  True, False],
       [False, False, False, False, False, False,  True]])>

In [90]:
val_if_large = max_exact + tf.dtypes.cast(
    tf.math.log(tf.cast(n, tf.float32) / max_exact) /
    math.log(max_distance / max_exact) * (num_buckets - max_exact),
    tf.int32,
)

val_if_large

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[-2147483647,           1,           2,           2,           3,
                  3,           3],
       [          1, -2147483647,           1,           2,           2,
                  3,           3],
       [          2,           1, -2147483647,           1,           2,
                  2,           3],
       [          2,           2,           1, -2147483647,           1,
                  2,           2],
       [          3,           2,           2,           1, -2147483647,
                  1,           2],
       [          3,           3,           2,           2,           1,
        -2147483647,           1],
       [          3,           3,           3,           2,           2,
                  1, -2147483647]], dtype=int32)>

In [91]:
val_if_large = tf.math.minimum(val_if_large, num_buckets - 1)
val_if_large

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[-2147483647,           1,           2,           2,           2,
                  2,           2],
       [          1, -2147483647,           1,           2,           2,
                  2,           2],
       [          2,           1, -2147483647,           1,           2,
                  2,           2],
       [          2,           2,           1, -2147483647,           1,
                  2,           2],
       [          2,           2,           2,           1, -2147483647,
                  1,           2],
       [          2,           2,           2,           2,           1,
        -2147483647,           1],
       [          2,           2,           2,           2,           2,
                  1, -2147483647]], dtype=int32)>

In [92]:
ret += tf.where(is_small, n, val_if_large)
ret

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[0, 4, 5, 5, 5, 5, 5],
       [1, 0, 4, 5, 5, 5, 5],
       [2, 1, 0, 4, 5, 5, 5],
       [2, 2, 1, 0, 4, 5, 5],
       [2, 2, 2, 1, 0, 4, 5],
       [2, 2, 2, 2, 1, 0, 4],
       [2, 2, 2, 2, 2, 1, 0]], dtype=int32)>

In [ ]:
_embed_init = None

self._relative_attention_bias = self.add_weight(
    "rel_embedding",
    shape=[self.relative_attention_num_buckets, self.num_heads],
    initializer=self._embed_init,
    dtype=self.dtype,
    trainable=True)

In [ ]:
values = tf.nn.embedding_lookup(_relative_attention_bias, rp_bucket)
values

In [ ]:
values = tf.expand_dims(
    tf.transpose(values, [2, 0, 1]),
    axis=0)  # shape (1, num_heads, qlen, klen)

values

In [ ]:
values = tf.tile(values, [batch_size, 1, 1, 1])
values

In [123]:
relative_position

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6],
       [-1,  0,  1,  2,  3,  4,  5],
       [-2, -1,  0,  1,  2,  3,  4],
       [-3, -2, -1,  0,  1,  2,  3],
       [-4, -3, -2, -1,  0,  1,  2],
       [-5, -4, -3, -2, -1,  0,  1],
       [-6, -5, -4, -3, -2, -1,  0]], dtype=int32)>

In [122]:
tf.math.sign(relative_position)

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[ 0,  1,  1,  1,  1,  1,  1],
       [-1,  0,  1,  1,  1,  1,  1],
       [-1, -1,  0,  1,  1,  1,  1],
       [-1, -1, -1,  0,  1,  1,  1],
       [-1, -1, -1, -1,  0,  1,  1],
       [-1, -1, -1, -1, -1,  0,  1],
       [-1, -1, -1, -1, -1, -1,  0]], dtype=int32)>

In [155]:
batch_size, qlen = get_shape_list(query)[:2]
print('batch_size: {}'.format(batch_size))
print('qlen: {}'.format(qlen))

klen = get_shape_list(key)[1]
print('klen: {}'.format(klen))

context_position = tf.range(qlen)[:, None]
memory_position = tf.range(klen)[None, :]
relative_position = memory_position - context_position
relative_position

batch_size: 1
qlen: 7
klen: 7


<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6],
       [-1,  0,  1,  2,  3,  4,  5],
       [-2, -1,  0,  1,  2,  3,  4],
       [-3, -2, -1,  0,  1,  2,  3],
       [-4, -3, -2, -1,  0,  1,  2],
       [-5, -4, -3, -2, -1,  0,  1],
       [-6, -5, -4, -3, -2, -1,  0]], dtype=int32)>

In [177]:
relative_attention_max_distance = 4
relative_attention_num_buckets = 5

def make_log_bucket_position(relative_pos, bucket_size, max_position):
    sign = tf.math.sign(relative_pos)
    mid = bucket_size // 2
    abs_pos = tf.where((relative_pos < mid) & (relative_pos > -mid), mid - 1, tf.math.abs(relative_pos))
    log_pos = (tf.math.ceil(tf.cast(tf.math.log(abs_pos / mid), tf.float32) / 
                            tf.math.log((max_position - 1) / mid) * (mid - 1)) 
                            + mid )
    bucket_pos = tf.cast(tf.where(abs_pos <= mid, 
                                  tf.cast(relative_pos, tf.float32), 
                                  log_pos * tf.cast(sign, tf.float32)), tf.int32)
    return bucket_pos

bucket_pos = make_log_bucket_position(relative_pos=relative_position, 
                                      bucket_size=relative_attention_num_buckets, 
                                      max_position=relative_attention_max_distance)

bucket_pos

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  5],
       [-1,  0,  1,  2,  3,  4,  5],
       [-2, -1,  0,  1,  2,  3,  4],
       [-3, -2, -1,  0,  1,  2,  3],
       [-4, -3, -2, -1,  0,  1,  2],
       [-5, -4, -3, -2, -1,  0,  1],
       [-5, -5, -4, -3, -2, -1,  0]], dtype=int32)>

In [191]:
relative_attention_max_distance = 6
relative_attention_num_buckets = relative_attention_max_distance

def make_log_bucket_position(relative_pos, bucket_size, max_position):
    sign = tf.math.sign(relative_pos)
    mid = bucket_size // 2
    abs_pos = tf.where((relative_pos < mid) & (relative_pos > -mid), mid - 1, tf.math.abs(relative_pos))
    log_pos = (tf.math.ceil(tf.cast(tf.math.log(abs_pos / mid), tf.float32) / 
                            tf.math.log((max_position - 1) / mid) * (mid - 1)) 
                            + mid )
    bucket_pos = tf.cast(tf.where(abs_pos <= mid, 
                                  tf.cast(relative_pos, tf.float32), 
                                  log_pos * tf.cast(sign, tf.float32)), tf.int32)
    return bucket_pos

bucket_pos = make_log_bucket_position(relative_pos=relative_position, 
                                      bucket_size=relative_attention_num_buckets, 
                                      max_position=relative_attention_max_distance)

bucket_pos

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  5,  6,  6],
       [-1,  0,  1,  2,  3,  5,  6],
       [-2, -1,  0,  1,  2,  3,  5],
       [-3, -2, -1,  0,  1,  2,  3],
       [-5, -3, -2, -1,  0,  1,  2],
       [-6, -5, -3, -2, -1,  0,  1],
       [-6, -6, -5, -3, -2, -1,  0]], dtype=int32)>

In [136]:
rel_pos_ids = bucket_pos[:qlen, :]
rel_pos_ids = tf.expand_dims(rel_pos_ids, axis=0)
rel_pos_ids

<tf.Tensor: shape=(1, 7, 7), dtype=int32, numpy=
array([[[ 0,  1,  2,  3,  4,  5,  5],
        [-1,  0,  1,  2,  3,  4,  5],
        [-2, -1,  0,  1,  2,  3,  4],
        [-3, -2, -1,  0,  1,  2,  3],
        [-4, -3, -2, -1,  0,  1,  2],
        [-5, -4, -3, -2, -1,  0,  1],
        [-5, -5, -4, -3, -2, -1,  0]]], dtype=int32)>

In [ ]:
att_span = relative_attention_max_distance
rel_embeddings = tf.expand_dims(
    rel_embeddings[relative_attention_max_distance - att_span : relative_attention_max_distance + att_span, :], 0
)

In [ ]:
relative_pos = build_relative_position(q, key_layer.size(-2), bucket_size = self.position_buckets, \
                                       max_position = self.max_relative_positions, device=query_layer.device)

In [23]:
from typing import List, Optional, Union

def shape_list(tensor: Union[tf.Tensor, np.ndarray]) -> List[int]:
    """
    Deal with dynamic shape in tensorflow cleanly.

    Args:
        tensor (`tf.Tensor` or `np.ndarray`): The tensor we want the shape of.

    Returns:
        `List[int]`: The shape of the tensor as a list.
    """
    if isinstance(tensor, np.ndarray):
        return list(tensor.shape)

    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

In [ ]:
# 5 x 6

In [63]:
class RelativePositionBias(tf.keras.layers.Layer):

    def __init__(self, d_model, num_buckets, max_relative_pos, bidirectional, **kwargs):
        super(RelativePositionBias, self).__init__(**kwargs)
        self.d_model = d_model
        self.num_buckets = num_buckets
        self.max_relative_pos = max_relative_pos

        self.bidirectional = bidirectional

    def build(self, input_shape):
        self.rel_embeddings = self.add_weight(shape=(self.num_buckets, self.d_model),
                                             #initializer=self.init_freq,
                                             trainable=True,
                                             name='rel_emb')


    def call(self, q, k, log_bucket=True):
        batch_size, qlen = get_shape_list(q)[:2] 
        klen = get_shape_list(k)[1]
        relative_pos = self.get_rel_pos(qlen, klen)
        if log_bucket:
            relative_pos = self.make_log_bucket_position(relative_pos)
        relative_pos = relative_pos[:qlen, :]
        relative_pos = tf.expand_dims(relative_pos, axis=0)
    
        print('relative_pos:', relative_pos)
        
        shape_list_pos = shape_list(relative_pos)
        print('shape_list_pos:', shape_list_pos)

        if len(shape_list_pos) == 2:
            relative_pos = tf.expand_dims(tf.expand_dims(relative_pos, 0), 0)
        elif len(shape_list_pos) == 3:
            relative_pos = tf.expand_dims(relative_pos, 1)
        # bsz x height x query x key
        elif len(shape_list_pos) != 4:
            raise ValueError(f"Relative position ids must be of dim 2 or 3 or 4. {len(shape_list_pos)}")

        print('relative_pos:', relative_pos)

        att_span = self.max_relative_pos
        print('1. self.rel_embeddings:', self.rel_embeddings)
        
        self.rel_embeddings = tf.expand_dims(
            self.rel_embeddings[self.max_relative_pos - att_span : self.max_relative_pos + att_span, :], 0
        )

        print('2. self.rel_embeddings:', self.rel_embeddings)

    def get_rel_pos(self, qlen, klen):
        context_position = tf.range(qlen)[:, None]
        memory_position = tf.range(klen)[None, :]
        relative_pos = memory_position - context_position
        return relative_pos


    def make_log_bucket_position(self, relative_pos):
        sign = tf.math.sign(relative_pos)
        mid = self.num_buckets // 2
        abs_pos = tf.where((relative_pos < mid) & (relative_pos > -mid), mid - 1, tf.math.abs(relative_pos))
        log_pos = (
            tf.math.ceil(
                tf.cast(tf.math.log(abs_pos / mid), tf.float32) / tf.math.log((self.max_relative_pos - 1) / mid) * (mid - 1)
            )
            + mid
        )
        bucket_pos = tf.cast(
            tf.where(abs_pos <= mid, tf.cast(relative_pos, tf.float32), log_pos * tf.cast(sign, tf.float32)), tf.int32
        )
        return bucket_pos


In [64]:
wq = tf.keras.layers.Dense(d_model, name='WQ')
wk = tf.keras.layers.Dense(d_model, name='WK')

query = wq(toy_lc_flux)
key = wk(toy_lc_flux)
query

<tf.Tensor: shape=(1, 7, 6), dtype=float32, numpy=
array([[[-0.4049514 , -0.54453087,  4.6461005 , -1.4279401 ,
         -4.5876546 , -3.1889935 ],
        [-0.4031911 , -0.54216385,  4.6259046 , -1.421733  ,
         -4.5677123 , -3.175131  ],
        [-0.40591156, -0.545822  ,  4.657117  , -1.4313259 ,
         -4.598532  , -3.196555  ],
        [-0.41247275, -0.5546447 ,  4.7323947 , -1.4544619 ,
         -4.672863  , -3.248224  ],
        [-0.42919567, -0.5771317 ,  4.9242606 , -1.5134304 ,
         -4.8623157 , -3.3799171 ],
        [-0.4055115 , -0.54528403,  4.652527  , -1.4299152 ,
         -4.594     , -3.1934042 ],
        [-0.39799017, -0.53517026,  4.566233  , -1.4033935 ,
         -4.5087914 , -3.1341739 ]]], dtype=float32)>

In [65]:
rel_pos_bias = RelativePositionBias(d_model=6, 
                                    num_buckets=5, max_relative_pos=5, 
                                    bidirectional=True)

values = rel_pos_bias(query, key, log_bucket=False)
values

relative_pos: tf.Tensor(
[[[ 0  1  2  3  4  5  6]
  [-1  0  1  2  3  4  5]
  [-2 -1  0  1  2  3  4]
  [-3 -2 -1  0  1  2  3]
  [-4 -3 -2 -1  0  1  2]
  [-5 -4 -3 -2 -1  0  1]
  [-6 -5 -4 -3 -2 -1  0]]], shape=(1, 7, 7), dtype=int32)
shape_list_pos: [1, 7, 7]
relative_pos: tf.Tensor(
[[[[ 0  1  2  3  4  5  6]
   [-1  0  1  2  3  4  5]
   [-2 -1  0  1  2  3  4]
   [-3 -2 -1  0  1  2  3]
   [-4 -3 -2 -1  0  1  2]
   [-5 -4 -3 -2 -1  0  1]
   [-6 -5 -4 -3 -2 -1  0]]]], shape=(1, 1, 7, 7), dtype=int32)
1. self.rel_embeddings: <tf.Variable 'relative_position_bias_10/rel_emb:0' shape=(5, 6) dtype=float32, numpy=
array([[ 0.04249322, -0.02939612,  0.06477737, -0.50271374, -0.02651012,
        -0.6698197 ],
       [-0.00114155, -0.5151731 , -0.70630115,  0.11261612,  0.6107096 ,
        -0.41813314],
       [ 0.12850839, -0.55583274,  0.30220467,  0.23526502, -0.17055827,
        -0.5858531 ],
       [-0.22770166,  0.47143584, -0.55156696, -0.24672067,  0.1727817 ,
         0.68841785],
       

In [55]:
max_position = 5
bucket_size = 5

context_position = tf.range(qlen)[:, None]
memory_position = tf.range(klen)[None, :]
relative_pos = memory_position - context_position
relative_pos

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6],
       [-1,  0,  1,  2,  3,  4,  5],
       [-2, -1,  0,  1,  2,  3,  4],
       [-3, -2, -1,  0,  1,  2,  3],
       [-4, -3, -2, -1,  0,  1,  2],
       [-5, -4, -3, -2, -1,  0,  1],
       [-6, -5, -4, -3, -2, -1,  0]], dtype=int32)>

In [56]:
relative_pos = tf.clip_by_value(relative_pos, -max_position + 1, max_position - 1) + max_position
relative_pos

<tf.Tensor: shape=(7, 7), dtype=int32, numpy=
array([[5, 6, 7, 8, 9, 9, 9],
       [4, 5, 6, 7, 8, 9, 9],
       [3, 4, 5, 6, 7, 8, 9],
       [2, 3, 4, 5, 6, 7, 8],
       [1, 2, 3, 4, 5, 6, 7],
       [1, 1, 2, 3, 4, 5, 6],
       [1, 1, 1, 2, 3, 4, 5]], dtype=int32)>

In [59]:
relative_pos = tf.range(-max_position, max_position, dtype=tf.float32)
sign = tf.sign(relative_pos)
mid = bucket_size // 2
abs_pos = tf.where((relative_pos < mid) & (relative_pos > -mid), mid - 1, tf.abs(relative_pos))
log_pos = tf.math.ceil(tf.math.log(abs_pos / mid) / tf.math.log((max_position - 1) / mid) * (mid - 1)) + mid
bucket_dict = tf.where(abs_pos <= mid, relative_pos, log_pos * sign)
bucket_dict

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([-4., -3., -3., -2., -1.,  0.,  1.,  2.,  3.,  3.], dtype=float32)>

In [61]:
for _ in range(relative_pos.shape.rank - 1):
    bucket_dict = tf.expand_dims(bucket_dict, 0)
    bucket_dict = tf.gather(bucket_dict, indices=tf.cast(relative_pos, tf.int32), axis=-1)

bucket_dict

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([-4., -3., -3., -2., -1.,  0.,  1.,  2.,  3.,  3.], dtype=float32)>

In [62]:
bucket_dict = bucket_dict[:qlen, :]
bucket_dict = tf.expand_dims(bucket_dict, axis=0)
bucket_dict

2023-10-05 13:05:43.578076: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at strided_slice_op.cc:117 : INVALID_ARGUMENT: Index out of range using input dim 1; input has only 1 dims


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} Index out of range using input dim 1; input has only 1 dims [Op:StridedSlice] name: strided_slice/

In [66]:
q_ids = tf.range(qlen, dtype=tf.int32)
k_ids = tf.range(klen, dtype=tf.int32)
rel_pos_ids = q_ids[:, None] - tf.tile(tf.reshape(k_ids, [1, -1]), [qlen, 1])
rel_pos_ids = rel_pos_ids[:qlen, :]
rel_pos_ids = tf.expand_dims(rel_pos_ids, axis=0)
rel_pos_ids

<tf.Tensor: shape=(1, 7, 7), dtype=int32, numpy=
array([[[ 0, -1, -2, -3, -4, -5, -6],
        [ 1,  0, -1, -2, -3, -4, -5],
        [ 2,  1,  0, -1, -2, -3, -4],
        [ 3,  2,  1,  0, -1, -2, -3],
        [ 4,  3,  2,  1,  0, -1, -2],
        [ 5,  4,  3,  2,  1,  0, -1],
        [ 6,  5,  4,  3,  2,  1,  0]]], dtype=int32)>

In [67]:
att_span = tf.cast(
            tf.minimum(
                tf.maximum(shape_list(query)[-2], shape_list(key)[-2]), max_position
            ),
            tf.int64,
        )

att_span

<tf.Tensor: shape=(), dtype=int64, numpy=5>

In [68]:
max_position - att_span

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [69]:
max_position + att_span

<tf.Tensor: shape=(), dtype=int64, numpy=10>

## PyTorch

In [71]:
import torch

In [72]:
class RelativePosition(torch.nn.Module):

    def __init__(self, num_units, max_relative_position):
        super(RelativePosition, self).__init__()
        self.num_units = num_units
        self.max_relative_position = max_relative_position
        self.embeddings_table = torch.nn.Parameter(torch.Tensor(max_relative_position * 2 + 1, num_units))
        torch.nn.init.xavier_uniform_(self.embeddings_table)

    def forward(self, length_q, length_k):
        range_vec_q = torch.arange(length_q)
        range_vec_k = torch.arange(length_k)
        distance_mat = range_vec_k[None, :] - range_vec_q[:, None]
        distance_mat_clipped = torch.clamp(distance_mat, -self.max_relative_position, self.max_relative_position)
        print(f"distance_mat_clipped: {distance_mat_clipped.shape}")
        final_mat = distance_mat_clipped + self.max_relative_position
        final_mat = torch.LongTensor(final_mat).cuda()
        print(f"final_mat: {final_mat.shape}")
        print(f"final_mat \n{final_mat}")
        embeddings = self.embeddings_table[final_mat].cuda()
    
        return embeddings

In [73]:
class MultiHeadAttentionLayer(torch.nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        self.max_relative_position = 2

        self.relative_position_k = RelativePosition(self.head_dim, self.max_relative_position)
        self.relative_position_v = RelativePosition(self.head_dim, self.max_relative_position)

        self.fc_q = torch.nn.Linear(hid_dim, hid_dim)
        self.fc_k = torch.nn.Linear(hid_dim, hid_dim)
        self.fc_v = torch.nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = torch.nn.Linear(hid_dim, hid_dim)
        
        self.dropout = torch.nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        len_k = key.shape[1]    #key = [batch_size, key_len, hid_dim]
        len_q = query.shape[1]  #query = [batch_size, query_len, hid_dim]
        len_v = value.shape[1]  #value = [batch_size, value_len, hid_dim]

        # Create Q, K, and V using input vectors
        query = self.fc_q(query)
        key = self.fc_k(key)
        value = self.fc_v(value)

        # Compute Attention scores
        r_q1 = query.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        r_k1 = key.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        attn1 = torch.matmul(r_q1, r_k1.permute(0, 1, 3, 2)) 

        r_q2 = query.permute(1, 0, 2).contiguous().view(len_q, batch_size*self.n_heads, self.head_dim)
        r_k2 = self.relative_position_k(len_q, len_k)
        attn2 = torch.matmul(r_q2, r_k2.transpose(1, 2)).transpose(0, 1)
        attn2 = attn2.contiguous().view(batch_size, self.n_heads, len_q, len_k)
        attn = (attn1 + attn2) / self.scale

        if mask is not None:
            attn = attn.masked_fill(mask==0, -1e10)

        # Convert attention scores into probability distributions
        attn = self.dropout(torch.softmax(attn, dim=-1))

        #attn = [batch size, n heads, query len, key len]
        r_v1 = value.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        weight1 = torch.matmul(attn, r_v1)
        r_v2 = self.relative_position_v(len_q, len_v)
        weight2 = attn.permute(2, 0, 1, 3).contiguous().view(len_q, batch_size*self.n_heads, len_k)
        weight2 = torch.matmul(weight2, r_v2)
        weight2 = weight2.transpose(0, 1).contiguous().view(batch_size, self.n_heads, len_q, self.head_dim)

        x = weight1 + weight2
        
        # x = [batch size, n heads, query len, head dim]
        x = x.permute(0, 2, 1, 3).contiguous()
        
        # x = [batch size, query len, n heads, head dim]
        x = x.view(batch_size, -1, self.hid_dim)
        
        # x = [batch size, query len, hid dim]
        x = self.fc_o(x)
        
        # x = [batch size, query len, hid dim]
        return x

## TensorFlow

In [ ]:
from typing import List, Optional, Union

def shape_list(tensor: Union[tf.Tensor, np.ndarray]) -> List[int]:
    """
    Deal with dynamic shape in tensorflow cleanly.

    Args:
        tensor (`tf.Tensor` or `np.ndarray`): The tensor we want the shape of.

    Returns:
        `List[int]`: The shape of the tensor as a list.
    """
    if isinstance(tensor, np.ndarray):
        return list(tensor.shape)

    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]


class RelativePositionBias(tf.keras.layers.Layer):

    def __init__(self, d_model, num_buckets, max_relative_pos, bidirectional, **kwargs):
        super(RelativePositionBias, self).__init__(**kwargs)
        self.d_model = d_model
        self.num_buckets = num_buckets
        self.max_relative_pos = max_relative_pos

        self.bidirectional = bidirectional

    def build(self, input_shape):
        self.rel_embeddings = self.add_weight(shape=(self.num_buckets, self.d_model),
                                             #initializer=self.init_freq,
                                             trainable=True,
                                             name='rel_emb')


    def call(self, q, k, log_bucket=True):
        batch_size, qlen = get_shape_list(q)[:2] 
        klen = get_shape_list(k)[1]
        relative_pos = self.get_rel_pos(qlen, klen) # 200 x 200
        if log_bucket:
            relative_pos = self.make_log_bucket_position(relative_pos)
        relative_pos = relative_pos[:qlen, :]
        relative_pos = tf.expand_dims(relative_pos, axis=0)
    
        print('relative_pos:', relative_pos)
        
        shape_list_pos = shape_list(relative_pos)
        print('shape_list_pos:', shape_list_pos)

        if len(shape_list_pos) == 2:
            relative_pos = tf.expand_dims(tf.expand_dims(relative_pos, 0), 0)
        elif len(shape_list_pos) == 3:
            relative_pos = tf.expand_dims(relative_pos, 1)
        # bsz x height x query x key
        elif len(shape_list_pos) != 4:
            raise ValueError(f"Relative position ids must be of dim 2 or 3 or 4. {len(shape_list_pos)}")

        print('relative_pos:', relative_pos)

        att_span = self.max_relative_pos
        print('1. self.rel_embeddings:', self.rel_embeddings)
        
        self.rel_embeddings = tf.expand_dims(
            self.rel_embeddings[self.max_relative_pos - att_span : self.max_relative_pos + att_span, :], 0
        )

        print('2. self.rel_embeddings:', self.rel_embeddings)

    def get_rel_pos(self, qlen, klen):
        context_position = tf.range(qlen)[:, None]
        memory_position = tf.range(klen)[None, :]
        relative_pos = memory_position - context_position
        return relative_pos


    def make_log_bucket_position(self, relative_pos):
        sign = tf.math.sign(relative_pos)
        mid = self.num_buckets // 2
        abs_pos = tf.where((relative_pos < mid) & (relative_pos > -mid), mid - 1, tf.math.abs(relative_pos))
        log_pos = (
            tf.math.ceil(
                tf.cast(tf.math.log(abs_pos / mid), tf.float32) / tf.math.log((self.max_relative_pos - 1) / mid) * (mid - 1)
            )
            + mid
        )
        bucket_pos = tf.cast(
            tf.where(abs_pos <= mid, tf.cast(relative_pos, tf.float32), log_pos * tf.cast(sign, tf.float32)), tf.int32
        )
        return bucket_pos

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None, return_mask=False):

	matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

	# scale matmul_qk
	dk = tf.cast(tf.shape(k)[-1], tf.float32)
	scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

	if mask is not None:
		print('Using mask...')
		steps = tf.shape(scaled_attention_logits)[2]
		mask_rshp = tf.tile(mask, [1,1,steps])
		mask_rshp += tf.transpose(mask_rshp, [0,2,1])
		mask_rshp = tf.minimum(1., mask_rshp)
		mask_rshp = tf.expand_dims(mask_rshp, 1)
		scaled_attention_logits += (mask_rshp*-1e9)

	# softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
	attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1, name='MaskedSoftMax')  # (..., seq_len_q, seq_len_k)
	output = tf.matmul(attention_weights, v, name='Z')  # (..., seq_len_q, depth_v)
	if return_mask:
		return output, attention_weights, mask_rshp
	return output, attention_weights


class HeadAttentionMulti(tf.keras.layers.Layer):
	def __init__(self, head_dim, num_heads):
		super(HeadAttentionMulti, self).__init__()
		self.num_heads = num_heads
		self.head_dim = head_dim
		
		self.d_model = self.num_heads * self.head_dim
		self.depth = self.d_model // self.num_heads # final dimension
		
		self.wq = tf.keras.layers.Dense(self.d_model, name='WQ')
		self.wk = tf.keras.layers.Dense(self.d_model, name='WK')
		self.wv = tf.keras.layers.Dense(self.d_model, name='WV')
		self.dense = tf.keras.layers.Dense(self.d_model, name='attmerge')

		#self.scaled_dot_product_attention = {
		#	'normal': scaled_dot_product_attention
		#}

	def split_heads(self, x, batch_size, name='qkv'):
		"""Split the last dimension into (num_heads, depth).
		Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
		"""
		x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
		return tf.transpose(x, perm=[0, 2, 1, 3], name=name)

	def call(self, x, training, mask=None):
		batch_size = tf.shape(x)[0]

		q = self.wq(x)  # (batch_size, seq_len, d_model)
		k = self.wk(x)  # (batch_size, seq_len, d_model)
		v = self.wv(x)  # (batch_size, seq_len, d_model)

		q = self.split_heads(q, batch_size, name='Q')  # (batch_size, num_heads, seq_len_q, depth)
		k = self.split_heads(k, batch_size, name='K')  # (batch_size, num_heads, seq_len_k, depth)
		v = self.split_heads(v, batch_size, name='V')  # (batch_size, num_heads, seq_len_v, depth)

		# scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
		# attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
		scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask=mask)

		scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

		concat_attention = tf.reshape(scaled_attention,
										(batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

		output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
		
		return output, attention_weights